# How to use and manipulate CSV files in Python

## Part 1 - What is a CSV file

A csv file is a type of file where the data is structured into rows and columns, using commas and new lines. They're commonly used to represent data in a spreadsheet - such as from Microsoft Excel or Google Sheets.

For example, if I had this spreadsheet on Google Sheets or Excel

![http://i.imgur.com/9ig18Kt.png](http://i.imgur.com/9ig18Kt.png)

And then I went to File->Download As->Comma-separated values and downloaded the file, it would look like this:

In this example, the first row has three things separated by commas. For each of the following rows, the first item correspons to the first item in the first row (so Box of cookies is an item, 2 is a quantity, and 4 is a price per item. 

P.S. If you can't remember which are rows and which are columns, you can think of columns like the roman columns (going up and down), and rows like running (left to right).

This shows probably the biggest reason we care about CSV files. They're structured, so they're easy to read programmatically, but they're also easy to just give to someone who knows nothing about code, so they can just open it in their favorite spreadsheet program. Many times when downloading large data sets from online, they will give you CSV files to read.

## Part 2 - Reading CSV Files

### Part 2a - Reading them manually

Remember that CSV files are just regular files with a standard formatting.
Therefore, you can just read them like a regular file, and get the data you want.

For example, if we want to figure out how much we will have to pay in the end, we have to go through the rows, and multiply the price by the quantity, and add them all up.

In [ ]:
csv_text = open("shopping_list.csv").read() # How to turn a file into a string
csv_text

In [ ]:
# Now we have to split it on the new lines, so each 
csv_text_split = csv_text.split("\n")
print(csv_text_split)

In [ ]:
# We can get rid of the first line, since that's just the headers. 
# We know that quantity is index 1, and price is index 2.
csv_text_split.pop(0) # Delete the line at index 0 - since it's the header

In [ ]:
# And now we just iterate through each line, and take out the information we want. 
total_price = 0
for line in csv_text_split: # For each line
    new_line = line.split(",") # split the line on the comma, so the line is now a list [Item,Quantity,Price]
    quantity = int(new_line[1]) # turn the thing at index 1 into an int
    price_per_item = int(new_line[2]) # take the dollar sign off the thing at index 2, and turn it into an int
    
    total_price += quantity*price_per_item

print("Total Price: ${}".format(total_price)) 

### Part 2b - Using the CSV library

That wasn't awful - but there's a lot of code in there that would be repeated in all CSVs. In addition, our code doesn't handle some special cases (What if there are commas in the item, for example?)

Because of this, Python comes with a CSV library that makes it extremely easy to turn a CSV file into a list of lists, so that you can parse it more easily. Let me show you how it works:

In [ ]:
# First step: import the library
import csv

# Second step: pass the open file into csv.reader
csv_lists = csv.reader(open("shopping_list.csv"))

# Third step: iterate through the file you created:
for line in csv_lists: 
    print(line)

## CSV1
Add a few lines ot the shopping_list.csv file using a code editor (e.g. atom) or a spreadsheet program (e.g. excel, but then store it as a CSV). Then execute the code above to print out the lines of data from the file.

We can find the total cost by looking at each line and multiplying the 2nd and 3rd values, quantity times price.

In [ ]:
csv_lists = csv.reader(open("shopping_list.csv"))
total_cost = 0
next(csv_lists) # this advances the csv_lists by one, skipping over the header line

costs = [int(line[2])* int(line[1]) for line in csv_lists]  # why do we need the int(...)'s
print(costs)

print('the total cost is $',sum(costs))

Something to keep in mind is that csv.reader isn't exactly a list of lists. It just goes through each line, and then becomes empty. So you can't read from csv_lists twice - the second time it will just be empty. This also means you can't do indexing on it. 

In [ ]:
print("here are the items in the csv_lists variable")
for line in csv_lists:
    print(line)

The reason the csv library does this is in case you had a very large CSV file - this way, you don't have to store it all in memory, you can just read it line by line.

To read from it more than once, you can convert it into a list after reading it. This will store the entire list in your computer's memory, and allow you to use it like a list of lists. 

In [ ]:
csv_lists = list(csv.reader(open("shopping_list.csv")))
print(csv_lists)

print(csv_lists[1])

With it as a list, we can also use a List Comprehension to get the total price.

In [ ]:
csv_lists = csv_lists[1:] # Take off the headers

In [ ]:
totals = [int(line[1])*int(line[2]) for line in csv_lists]
print(totals)
print(sum(totals))

Now you have a list of lists, which is the data from your CSV.

Try writing code that goes through csv_lists, and prints out the item you're spending the most money on.

## CSV2
Write a function which finds the total number of items you are buying. Use a list comprehension to get the list of quantities of each item, then sum it.

In [ ]:
def total_num_of_items(csv_lists):
    pass

csv_lists = list(csv.reader(open("shopping_list.csv")))
print(total_num_of_items(csv_lists))

You might have noticed a lot of annoying things about working with this library while working with it. For one, you have to drop the first row, since it doesn't contain any data you want. Secondly, you have to refer to the items by index, which means you have to know the index of what you want.

These issues can be solved with the DictReader module of the csv library. Let me show you how that one works, and what it produces:

In [ ]:
# csv library is already imported
csv_file = csv.DictReader(open("shopping_list.csv"))
csv_list = list(csv_file)

# now let's see what's inside
for line in csv_list:
    print(line['Item'],line)
print(list(csv_list))

As you can see, the DictReader takes in a CSV file, and gives you a bunch of dictionaries, where the key is the header, and the value is the value at that line. This makes it easy to write very readable code, as you can use the name of the header to get what you want. For example, to rewrite the "total cost" code:

In [ ]:
csv_file = csv.DictReader(open("shopping_list.csv"))
total_cost = 0
for line in csv_file:
    quantity = int(line['Quantity'])
    price = int(line['Price per item in Dollars'])
    total_cost += quantity*price
print("Total cost is: ${}".format(total_cost))

The list comprehension version looks like this

In [ ]:
csv_file = csv.DictReader(open("shopping_list.csv"))
print(sum(
    [int(line['Quantity'])*int(line['Price per item in Dollars']) for line in csv_file]
))

It's up to you which version you want to use - whatever you're more comfortable with and you think looks the best.

## Part 3: Writing a CSV file

Like reading CSV files, we don't necessarily need the CSV library to create a CSV. However, It makes it a lot easier. In fact, I will only go over how to create one using the CSV library.

Let's say we want to create a CSV of the first 100 numbers, and their values at x^2, x^3, and sqrt(x)
Just like there's a csv.reader and a csv.DictReader, there's also a csv.writer and a csv.DictWriter. I'll show both ways of using them.

This also shows the "with" method of opening a file.

In [ ]:
import math
# First way, using CSV writer
with open("number_values.csv", "w") as new_csv: # we add the extra 'w' parameter for saying this file will be written to
    writer = csv.writer(new_csv)
    writer.writerow(["Number", "Number Squared", "Number Cubed", "Square Root of Number"])
    for i in range(1,101):
        writer.writerow([i, i**2, i**3, round(math.sqrt(i),2)])
# Notice there's no "close" statement

In [ ]:
with open("number_values.csv") as f:
    print(f.read())

The other way to do this is to use a DictWriter - I'll show you how to do that below.
Remember that the way the DictWriter worked was that each line was a dictionary mapping the header to its value at that line. The writer will work similarly, for each line, we will write a dictionary.

In [ ]:
with open("number_values2.csv", "w") as new_csv:
    # we have to tell the writer what our top fields are
    writer = csv.DictWriter(new_csv, fieldnames=["num", "squared", "cubed", "sqrt"])
    writer.writeheader() # to write the header
    for i in range(1,101):
        writer.writerow({"num": i, "squared": i**2, "cubed": i**3, "sqrt": round(math.sqrt(i), 2)})
with open("number_values2.csv") as f:
    print(f.read())
    

As you see, they produce the same output. It's up to you which one you want to do, depending on the type of CSV file you're trying to read/write from.

## Real Data



### Real Estate data

Download this CSV file: http://samplecsvs.s3.amazonaws.com/Sacramentorealestatetransactions.csv

Found from here: https://support.spatialkey.com/spatialkey-sample-csv-data/

"The Sacramento real estate transactions file is a list of 985 real estate transactions in the Sacramento area reported over a five-day period, as reported by the Sacramento Bee. Note that this file has address level information that you can choose to geocode, or you can use the existing latitude/longitude in the file."

We've already downloaded it into the file RE.csv in this folder, but you can download it again if you want. You should open it in excel or googlesheets and look at the data as a spreadsheet.

Next we look at the data using Python, and observe that it has 12 columns of data.

In [ ]:
csv_file = csv.DictReader(open('RE.csv'))
csv_list = list(csv_file)

for row in csv_list:
    pass#print(row)


Finish these functions - the parameter will be a string which is the file name. We solve the first one for you!

In [ ]:
# What is the average price of house sold?
def average_price(csv_filename):
    csv_file = csv.DictReader(open(csv_filename))
    prices = [int(line['price']) for line in csv_file]
    return( sum(prices)/len(prices))

print('the average price is',average_price("RE.csv"))


In [ ]:
# return a list of all house with at least the specified number of bedroom and under the specified price
def find_house(bedrooms,max_price):
    pass

z = OrderedDict([('street', '3526 HIGH ST'), ('city', 'SACRAMENTO'), ('zip', '95838'), ('state', 'CA'), ('beds', '2'), ('baths', '1'), ('sq__ft', '836'), ('type', 'Residential'), ('sale_date', 'Wed May 21 00:00:00 EDT 2008'), ('price', '59222'), ('latitude', '38.631913'), ('longitude', '-121.434879')])
print(z)

In [ ]:
# What is the most expensive house?
def price_of_most_expensive_house(csv_file):
    # first find the prices then take the max and return it
    pass # write your code here

In [ ]:
# Which zipcodes have the most expensive house sold?
def addresses_of_most_expensive_houses(csv_file):
    # first find the price of the most expensive house (by calling previous function!)
    # then find the addresses of the houses that cost M using a list comprehension
    # then return that list
    pass # write your code here

In [ ]:
# Which house had the highest ratio of square feet to price
# In other words: which house was the most expensive per square foot?
# Return the address of the house
def most_expensive_house_per_sq_ft(csv_file):
    pass # write your code here
    # hint: find the list of prices per square foot using a list comprehension
    # then find the max of those values
    # then using another list comprehension to find the address of the house with the highest cost/sqft
    # and return that

### Number 2:

Download an interesting CSV file online, and write code to find an interesting fact about it!

Here is an example of somewhere you can get an interesting CSV file: https://catalog.data.gov/dataset?res_format=CSV